In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15, 'lines.linewidth': 3, 'savefig.dpi': 1200})

import os, random, time, pickle, torch
import numpy as np

In [2]:
from blurnet import trainer

job = trainer.BlurJob('save', 'vision_datasets')
completed_configs = job.overview()

19 works completed
average processing time 34m11.45s


In [3]:
from jarvis.utils import grouping

groups = grouping(completed_configs)
losses, accs = {}, {}
for key, configs in groups.items():
    losses[key] = []
    accs[key] = []
    for config in configs:
        w_id = job.configs.fetch_id(config)
        preview = job.previews.fetch_record(w_id)
        losses[key].append(preview['loss_test'])
        accs[key].append(preview['acc_test'])

In [4]:
sorted_keys = [key for key, _ in sorted([(key, np.mean(losses[key])) for key in groups], key=lambda item:item[1])]
print('groups sorted according to mean loss')
for key in sorted_keys:
    print(f'\n{key}')
    print('loss: {:.3f} ({:.3f})'.format(np.mean(losses[key]), np.std(losses[key])/len(losses[key])**0.5))
    print('acc: {:.2%} ({:.2%})'.format(np.mean(accs[key]), np.std(accs[key])/len(accs[key])**0.5))

groups sorted according to mean loss

{'model_config': {'blur_sigma': None}}
loss: 0.235 (0.002)
acc: 92.50% (0.09%)

{'model_config': {'blur_sigma': 0.5}}
loss: 0.249 (0.003)
acc: 91.90% (0.13%)

{'model_config': {'blur_sigma': 1.5}}
loss: 0.358 (0.015)
acc: 88.14% (0.55%)

{'model_config': {'blur_sigma': 2.5}}
loss: 0.512 (0.007)
acc: 82.86% (0.33%)


In [ ]:
key = sorted_keys[2]
print(f'export {len(groups[key])} models with\n{key}\n')

for config in groups[key]:
    w_id = job.configs.fetch_id(config)
    print(f'exporting {w_id}...')
    job.export(w_id, 'save/exported')

export 5 models with
{'model_config': {'blur_sigma': 1.5}}

exporting 03375596...
exporting 3A95D107...
exporting 3F26F60C...
